In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

Saving sherlock-holm.es_stories_plain-text_advs.txt to sherlock-holm.es_stories_plain-text_advs.txt


In [3]:
# Read the text file
with open(path_to_file, 'r', encoding='utf-8') as file:
  text = file.read()

In [4]:
import tensorflow as tf

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [6]:
#Now let's tokenize the text to create a sequence of words:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [7]:
#In the above code, the text is tokenized, which means it is divided into individual words or tokens.
#The Tokenizer' object is created, which will handle the tokenization process.
#The 'fit_on_texts' method of the tokenizer is called, passing the 'text' as input.
#This method analyzes the text and builds a vocabulary of unique words, assigning each word a numerical index.
#The 'total_words' variable is then assigned the value of the length of the word index plus one, representing the total number of distinct words in the text.

In [8]:
#Now let's create input-output pairs by splitting the text into sequences of tokens and forming n-grams from the sequences:

In [9]:
input_sequences = []
for line in text.split('\n'):
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list )):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [10]:
# n the above code, the text data is split into lines using the '\n' character as a delimiter. For each line in the text, the 'texts_to_sequences'
# method of the tokenizer is used to convert the line into a sequence of numerical tokens based on the previously created vocabulary.
# The resulting token list is then iterated over using a for loop. For each iteration, a subsequence, or n-gram, of tokens is extracted, ranging from
# the beginning of the token list up to the current index 'i'.

In [11]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array (pad_sequences (input_sequences, maxlen=max_sequence_len, padding='pre'))

In [12]:
#In the above code, the input sequences are padded to ensure all sequences have the same length.
#The variable 'max_sequence_len' is assigned the maximum length among all the input sequences.
#The 'pad_sequences' function is used to pad or truncate the input sequences to match this maximum length.
#The 'pad_sequences' function takes the input_sequences list, sets the maximum length to 'max sequence_len',
#and specifies that the padding should be added at the beginning of each sequence using the 'padding=pre' argument. Finally,
#the input sequences are converted into a numpy array to facilitate further processing.
# Now let's split the sequences into input and output:

In [13]:
x = input_sequences[:, :-1]
y = input_sequences [:, -1]

In [14]:
# In the above code, the input sequences are split into two arrays, 'X' and 'y', to create the input and output for training the next word prediction model. The 'X' array is assigned the values of all rows in the 'input_sequences' array except for the last column. It means that 'X' contains all the tokens in each sequence except for the last one, representing the input context.

In [15]:
# On the other hand, the 'y' array is assigned the values of the last column in the 'input_sequences' array, which represents the target or predicted word.

In [16]:
# Now let's convert the output to one-hot encode vectors:

In [17]:
y= np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [18]:
# In the above code, we are converting the output array into a suitable format for training a model, where each target word is represented as a binary vector.
# Now let's build a neural network architecture to train the model:

In [19]:
model = Sequential()
model.add(Embedding (total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [20]:
# The code above defines the model architecture for the next word prediction model. The 'Sequential' model is created, which represents a linear stack of layers.
# The first layer added to the model is the 'Embedding' layer, which is responsible for converting the input sequences into dense vectors of fixed size. It takes three arguments:
# (1). 'total_words', which represents the total number of distinct words in the vocabulary;
# (2). '100', which denotes the dimensionality of the word embeddings;
# (3). and 'input length', which specifies the length of the input sequences.
# The next layer added is the 'LSTM' layer, a type of recurrent neural network (RNN) layer designed for capturing sequential dependencies in the data.
# It has 150 units, which means it will learn 150 internal representations or memory cells.
# Finally, the 'Dense' layer is added, which is a fully connected layer that produces the output predictions.
# It has 'total_words' units and uses the 'softmax' activation function to convert the predicted scores into probabilities, indicating
# the likelihood of each word being the next one in the sequence.
# Now let's compile and train the model:

In [ ]:
# Now let's compile and train the model:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 187s 62ms/step - loss: 6.2420 - accuracy: 0.0759
Epoch 2/100
3010/3010 [==============================] - 187s 62ms/step - loss: 5.5222 - accuracy: 0.1236
Epoch 3/100
3010/3010 [==============================] - 179s 60ms/step - loss: 5.1387 - accuracy: 0.1466
Epoch 4/100
3010/3010 [==============================] - 189s 63ms/step - loss: 4.8166 - accuracy: 0.1640
Epoch 5/100
3010/3010 [==============================] - 178s 59ms/step - loss: 4.5145 - accuracy: 0.1820
Epoch 6/100
3010/3010 [==============================] - 176s 58ms/step - loss: 4.2218 - accuracy: 0.2017
Epoch 7/100
3010/3010 [==============================] - 183s 61ms/step - loss: 3.9439 - accuracy: 0.2273
Epoch 8/100
3010/3010 [==============================] - 185s 61ms/step - loss: 3.6805 - accuracy: 0.2578
Epoch 9/100
3010/3010 [==============================] - 188s 62ms/step - loss: 3.4299 - accuracy: 0.2920
Epoch 10/100
3010/3010 [======================

In [ ]:
# In the above code, the model is being compiled and trained. The 'compile' method configures the model for training. The 'loss' parameter is set to 'categorical_crossentropy',
# a commonly used loss function for multi-class classification problems. The 'optimizer' parameter is set to 'adam', an optimization algorithm that adapts the learning rate during
# training.

In [ ]:
# The 'metrics' parameter is set to 'accuracy' to monitor the accuracy during training. After compiling the model, the 'fit' method is called to train the model on
# the input sequences 'X' and the corresponding output 'y'. The 'epochs' parameter specifies the number of times the training process will iterate over the entire dataset.
# The 'verbose' parameter is set to '1' to display the training process.

In [ ]:
# The above code will take more than an hour to execute. Once the code is executed, here's how we can generate the next word predictions using our model:

In [ ]:
#seed text = "I will leave if they"
seed_text = "Are you"
next_words=3

for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen = max_sequence_len-1, padding='pre')
  predicted= np.argmax(model.predict(token_list), axis=-1)
  output_word = ""
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word= word
      break
  seed_text += " " + output_word
print(seed_text)